# *sms ham & spam classification*

## 1st we know how countvectorizer work


**Here we going to make a sample data**

In [ ]:
# example text for model training
simple_train = ['call you tonight', 'Call me a cab', 'please call me.. please']

**import and instantiate CountVectorizer (with the default parameters)**

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer

#  instantiate CountVectorizer (vectorizer)
vect = CountVectorizer()

**When we fit CountVectorizer it will a vocabulary of word in a dict**

In [ ]:
# 3. fit
# learn the 'vocabulary' of the training data (occurs in-place)
vect.fit(simple_train)

In [ ]:
# examine the fitted vocabulary
vect.get_feature_names()

**know we are going to use transform function after fitiong the data**

**It will convert vocabulary to number(sparse matrix)**

In [ ]:
# 4. transform training data into a 'document-term matrix','sparse matrix'
simple_train_dtm = vect.transform(simple_train)
print(simple_train_dtm)

**Know we are going to convert sparse matrix to a dense matrix**

In [ ]:
simple_train_dtm.toarray()

examine the sparse matrix contents
* left: coordinates of non-zero values
* right: values at that point

CountVectorizer() will output a sparse matrix

In [ ]:
print('sparse matrix')
print(simple_train_dtm)

print('dense matrix')
print(simple_train_dtm.toarray())

In [ ]:
import pandas as pd
# examine the vocabulary and document-term matrix together
# pd.DataFrame(matrix, columns=columns)
pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names())

**By the above data frame you essaly understand**

**How we convert text data in to number**
**you can see we have three row in the data frame 
becouse there are only three sentances in over simple data**

**here you can see all the vocobulary we get from the data, now become a featue**


# Now be going to work on original data 

In [ ]:
#reading data
import warnings as wr
wr.filterwarnings("ignore")
import matplotlib.pyplot as plt
import seaborn as sns

sms=pd.read_csv("../input/sms-spam-collection-dataset/spam.csv")
sms.head()

In [ ]:
sms.shape

In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(sms.isnull(),yticklabels=False,cbar=True,cmap='mako')

In [ ]:
sms.isnull().sum()

Here we can say there are many NaN value in numamed so we going to drop those frature

In [ ]:
sms=sms.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis=1)
sms.head()

Feature description
* v1=label(ham/spam) outcome to predict
* v2=sms

In [ ]:
# examine the class distribution
sms.v1.value_counts()

In [ ]:

sns.countplot(sms["v1"])

In [ ]:
# convert label to a numerical variable

sms['v1'] = sms.v1.map({'ham':0, 'spam':1})

In [ ]:
sms.head()

**Balance Data**

In [ ]:
ham=sms[sms["v1"]==0]
spam=sms[sms["v1"]==1]

In [ ]:
ham.shape,spam.shape

In [ ]:
ham=ham.sample(spam.shape[0])
ham.shape

In [ ]:
data=spam.append(ham,ignore_index=True)
print("Shape :",data.shape)
data.head()

In [ ]:
sns.countplot(data["v1"])

* Now we can see data is balanced

**dividing data in X & Y**

In [ ]:
X = data.v2
y = data.v1

# Model building

In [ ]:
# split X and y into training and testing sets
# by default, it splits 75% training and 25% test
# random_state=1 for reproducibility

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

**calling CountVectorizer**

In [ ]:
vect = CountVectorizer()

In [ ]:
X_train_dtm = vect.fit_transform(X_train)

In [ ]:
X_train_dtm

In [ ]:
# 4. transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = vect.transform(X_test)
X_test_dtm

# you can see that the number of columns, 7456, is the same as what we have learned above in X_train_dtm

# MultinomialNB
**MultinomialNB is the one of the best algo for Text data**

In [ ]:
# 1. import
from sklearn.naive_bayes import MultinomialNB

# 2. instantiate a Multinomial Naive Bayes model
nb = MultinomialNB()

In [ ]:
# 3. train the model 

nb.fit(X_train_dtm, y_train)

In [ ]:
# 4. make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [ ]:
# calculate accuracy of class predictions
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

**Here we got .96 score**

**printing the confusion matrix**
to compare actual and prediction

In [ ]:

cf_matrix=metrics.confusion_matrix(y_test, y_pred_class)
cf_matrix

In [ ]:
import matplotlib.pyplot as plt
#Ploting confusion matrix
plt.figure(figsize=(8,5))
sns.heatmap(cf_matrix, annot=True, fmt='d')

In [ ]:
# print message text for the false positives (ham incorrectly classified as spam)

X_test[(y_pred_class==1) & (y_test==0)]

In [ ]:
# print message text for the false negatives (spam incorrectly classified as ham)
X_test[(y_pred_class==0) & (y_test==1)]

In [ ]:
# calculate AUC
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
metrics.roc_auc_score(y_test, y_pred_prob)

# LogisticRegression

In [ ]:
# 1. import
from sklearn.linear_model import LogisticRegression

# 2. instantiate a logistic regression model
logreg = LogisticRegression()

In [ ]:
# 3. train the model using X_train_dtm
logreg.fit(X_train_dtm, y_train)

In [ ]:
# 4. make class predictions for X_test_dtm
y_pred_class = logreg.predict(X_test_dtm)

In [ ]:
# calculate predicted probabilities for X_test_dtm (well calibrated)
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

In [ ]:
# calculate accuracy
metrics.accuracy_score(y_test, y_pred_class)

In [ ]:
# calculate AUC
metrics.roc_auc_score(y_test, y_pred_prob)

In [ ]:
# remove English stop words
vect1 = CountVectorizer(stop_words='english')

X_train_1 = vect1.fit_transform(X_train)

X_train_1


In [ ]:
# include 1-grams and 2-grams

# how to differentiate between "Happy", "Not Happy", "Very Happy"
vect2 = CountVectorizer(ngram_range=(1, 2))

X_train_2 = vect2.fit_transform(X_train)

X_train_2

In [ ]:
# ignore terms that appear in more than 50% of the documents
vect3 = CountVectorizer(max_df=0.5)

X_train_3 = vect3.fit_transform(X_train)

X_train_3

In [ ]:
# only keep terms that appear in at least 2 documents
vect4 = CountVectorizer(min_df=2)

X_train_4 = vect4.fit_transform(X_train)

X_train_4

In [ ]:
vect_combined= CountVectorizer(stop_words='english',ngram_range=(1, 2),min_df=2,max_df=0.5)

In [ ]:
X_train_c = vect_combined.fit_transform(X_train)
X_test_c = vect_combined.transform(X_test)

X_train_c

In [ ]:
# 1. import
from sklearn.naive_bayes import MultinomialNB

# 2. instantiate a Multinomial Naive Bayes model
nb = MultinomialNB()

nb.fit(X_train_c, y_train)

y_pred_class = nb.predict(X_test_c)

nb_cf_matrix=metrics.confusion_matrix(y_test, y_pred_class)

In [ ]:
plt.figure(figsize=(8,5))
sns.heatmap(nb_cf_matrix, annot=True, fmt='d')

# Please do a up vote if you find usefull
**write your suggestion in the comment**

### Thanks